# Get started

First we load ipython package sql
It can be installed by
```bash
pip install ipython-sql
```

In [105]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Connect to our database in *docker*

In [106]:
# PostgreSQL local
%sql postgresql://postgres@localhost:5432/datascience

In [107]:
%sql select madlib.version();

 * postgresql://postgres@localhost:5432/datascience
1 rows affected.


version
"MADlib version: 1.17.0, git revision: unknown, cmake configuration time: Sat Dec 19 21:45:13 UTC 2020, build type: RelWithDebInfo, build system: Linux-4.19.121-linuxkit, C compiler: gcc 5.4.0, C++ compiler: g++ 5.4.0"


## Load data
Create table `survey` to load data. All rows are integers

In [108]:
%%sql
DROP TABLE IF EXISTS survey;

create table survey(mydepv integer, 
                    price integer, 
                    income integer, 
                    age integer);

 * postgresql://postgres@localhost:5432/datascience
Done.
Done.


[]

Load from file. it was downloaded from [Hyper MEPhIx](https://hyper.mephi.ru/assets/courseware/v1/345e8b1c6ea11120575066ec4ac58f4a/asset-v1:MEPhIx+CS712DS+2020Fall+type@asset+block/survey.csv)


In [109]:
%%sql
COPY survey(mydepv, price, income, age)
FROM '/workdir/DataScience/Lab7/survey.csv'
DELIMITER ','
CSV HEADER;

 * postgresql://postgres@localhost:5432/datascience
750 rows affected.


[]

Check rows - Alright :)

In [110]:
%sql select * from survey limit 10;

 * postgresql://postgres@localhost:5432/datascience
10 rows affected.


mydepv,price,income,age
1,10,33,37
0,20,21,55
1,30,59,55
1,20,76,44
0,30,24,37
0,20,22,32
1,10,28,32
1,10,49,38
0,30,76,43
1,20,59,55


## Modify data
Add columns `price20` and `price30`

In [111]:
%%sql
alter table survey 
add column price20 integer;

 * postgresql://postgres@localhost:5432/datascience
Done.


[]

In [112]:
%%sql
alter table survey 
add column price30 integer;

 * postgresql://postgres@localhost:5432/datascience
Done.


[]

Set them as 1 or 0 for prices = 10 or 20 or 30

In [113]:
%%sql
update survey set price20=1 where price=20;
update survey set price20=0 where price!=20;
update survey set price30=1 where price=30;
update survey set price30=0 where price!=30;

 * postgresql://postgres@localhost:5432/datascience
250 rows affected.
500 rows affected.
250 rows affected.
500 rows affected.


[]

Check them

In [114]:
%sql select * from survey limit 10;

 * postgresql://postgres@localhost:5432/datascience
10 rows affected.


mydepv,price,income,age,price20,price30
1,30,59,55,0,1
0,30,24,37,0,1
0,30,76,43,0,1
0,30,45,32,0,1
0,30,21,46,0,1
0,30,49,44,0,1
0,30,31,32,0,1
0,30,22,32,0,1
0,30,29,32,0,1
1,30,59,55,0,1


Create table with columns of dependent and independent

In [115]:
%%sql
DROP TABLE IF EXISTS survey2;
CREATE TABLE survey2 (
    mydepv boolean,
    ind_values integer[]
);

 * postgresql://postgres@localhost:5432/datascience
Done.
Done.


[]

Insert values MYDEPV and array of others

In [116]:
%%sql
insert into survey2(mydepv, ind_values)
select survey.mydepv::bool, 
ARRAY[1, 
      survey.income, 
      survey.age, 
      survey.price20, 
      survey.price30] as arr from survey;

 * postgresql://postgres@localhost:5432/datascience
750 rows affected.


[]

Check them

In [117]:
%sql select * from survey2 limit 10;

 * postgresql://postgres@localhost:5432/datascience
10 rows affected.


mydepv,ind_values
True,"[1, 59, 55, 0, 1]"
False,"[1, 24, 37, 0, 1]"
False,"[1, 76, 43, 0, 1]"
False,"[1, 45, 32, 0, 1]"
False,"[1, 21, 46, 0, 1]"
False,"[1, 49, 44, 0, 1]"
False,"[1, 31, 32, 0, 1]"
False,"[1, 22, 32, 0, 1]"
False,"[1, 29, 32, 0, 1]"
True,"[1, 59, 55, 0, 1]"


## Create logistic regression with MADLib

In [118]:
%%sql
DROP TABLE IF EXISTS survey_logregr, survey_logregr_summary;
SELECT madlib.logregr_train('survey2',
              'survey_logregr',
              'mydepv',
              'ind_values');

 * postgresql://postgres@localhost:5432/datascience
Done.
1 rows affected.


logregr_train
""


### Summary

In [119]:
%%sql
SELECT unnest(array['intercept',
                    'income', 
                    'age', 
                    'price20', 
                    'price30']) as attribute,
       unnest(coef) as coefficient,
       unnest(std_err) as standard_error,
       unnest(z_stats) as z_stat,
       unnest(p_values) as pvalue,
       unnest(odds_ratios) as odds_ratio
    FROM survey_logregr;

 * postgresql://postgres@localhost:5432/datascience
5 rows affected.


attribute,coefficient,standard_error,z_stat,pvalue,odds_ratio
intercept,-6.02116057636787,0.532440923644448,-11.3085983983993,1.18974837145447e-29,0.00242685141358459
income,0.128759374924942,0.00923035837106702,13.9495531753723,3.16582525170149e-44,1.13741640032794
age,0.0350637796423411,0.0117900836733219,2.97400600486677,0.00293939322817078,1.03568576236067
price20,-0.744177494951807,0.264387873296036,-2.81471871487294,0.00488199444444116,0.475124931924531
price30,-2.21028046675832,0.311075548871486,-7.10528511410408,1.20074395698392e-12,0.109669885444447


Create results table with odds_ratio and prediction

In [120]:
%%sql
drop table if exists res_survey;
create table res_survey(mydepv integer, 
                        price integer, 
                        income integer, 
                        age integer,
                        price20 integer,
                        price30 integer,
                        odds_ratio real, 
                        prediction real);

 * postgresql://postgres@localhost:5432/datascience
Done.
Done.


[]

Put values to result table and count odds\_ratio as _coef1 + coef2\*param1 + coef3\*param2 + coef4\*param3 + coef5\*param4_

In [121]:
%%sql
insert into res_survey
select s.mydepv,
        s.price, 
        s.income, 
        s.age, 
        s.price20, 
        s.price30,
        (coef[1] + coef[2] * s.income + 
        coef[3] * s.age + 
        coef[4] * s.price20 + 
        coef[5] * s.price30) as odds_ratio,
        0 as prediction
from survey_logregr, survey as s;

 * postgresql://postgres@localhost:5432/datascience
750 rows affected.


[]

Count prediction as _exp(odds\_ratio) / (1 + exp(odds\_ratio))_

In [122]:
%%sql
update res_survey set prediction = (exp(odds_ratio) / (1+exp(odds_ratio)));

 * postgresql://postgres@localhost:5432/datascience
750 rows affected.


[]

Make predictions with coefs

In [123]:
%%sql
select exp_odds_ratio / (1 + exp_odds_ratio) as prediction
from
(select  exp(coef[1] + coef[2] * 19.114 + 
        coef[3] * 33 + 
        coef[4] * 1 + 
        coef[5] * 0) as exp_odds_ratio
from survey_logregr) as tmp;

 * postgresql://postgres@localhost:5432/datascience
1 rows affected.


prediction
0.0412046455698882


Count overall with \+ from predicted data

In [124]:
%%sql
select sum(mydepv) as mydepv, sum(prediction)::integer as predicted from res_survey;

 * postgresql://postgres@localhost:5432/datascience
1 rows affected.


mydepv,predicted
324,324


And finaly see our table

In [125]:
%%sql
select * from res_survey limit 10;

 * postgresql://postgres@localhost:5432/datascience
10 rows affected.


mydepv,price,income,age,price20,price30,odds_ratio,prediction
1,30,59,55,0,1,1.29387,0.784801
0,30,24,37,0,1,-3.84386,0.0209621
0,30,76,43,0,1,3.06201,0.955298
0,30,45,32,0,1,-1.31523,0.211613
0,30,21,46,0,1,-3.91456,0.0195591
0,30,49,44,0,1,-0.379425,0.406265
0,30,31,32,0,1,-3.11786,0.0423766
0,30,22,32,0,1,-4.27669,0.0136983
0,30,29,32,0,1,-3.37538,0.0330739
1,30,59,55,0,1,1.29387,0.784801
